# 3 - Spectral Degradation

**Requirements:**
- spectral irradiance (measured or simulated)
- wavelengths of spectral irradiance data
- module RH
- module temperature


**Objectives:**
1. Read in spectral irradiance
2. Calculate spectral degradation

In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.3.3

In [2]:
import os
import pandas as pd
import numpy as np
import pvdeg
from pvdeg import DATA_DIR

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)

Working on a  Windows 10
Python version  3.9.19 (main, May  6 2024, 20:12:36) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.2.3
pvdeg version  0.4.3.dev22+g0ef212c.d20241017


## 1. Read in spectral irradiance data

Spectral degradation has 4 main requirements:
- Spectral Irradiance [W/m^2 nm]
- Wavelength [nm] 
- Module Relative Humidity [%]
- Module Temperature [C]

For more advanced scenarios, you may want to calculate the degradation of a particular layer within the module. Below, we are using *backside* irradiance and therefore a slightly different temperature and humidity have been calculated. To calculate degradation on the backside, we used `pvdeg.humidity.rh_backsheet`. For the the front side, you should use `pvdeg.humidity.rh_surface_outside` or `rh_front_encap`


For this tutorial we are using pre-generated data from a ray-tracing simulation. To calculate the degradation rate, we will need the wavelengths used in the simulation. 

In [ ]:
wavelengths = np.array(range(280,420,20))

SPECTRA = pd.read_csv(os.path.join(DATA_DIR,'spectra.csv'), header=0, index_col=0)
SPECTRA.head()

,Spectra,Temperature,RH
timestamp,,,
2021-03-09 10:00:00,"[0.6892146677599185, 0.40215646494410884, 0.67...",45,59
2021-03-09 11:00:00,"[0.15575709102178648, 0.5464374649246564, 0.68...",44,56
2021-03-09 12:00:00,"[0.22782105874481207, 0.9056495270031296, 0.26...",59,39
2021-03-09 13:00:00,"[0.3741943134512433, 0.035830980984344674, 0.4...",44,13
2021-03-09 14:00:00,"[0.40321187996337626, 0.6473167864022122, 0.69...",25,39


**Convert To Expected Format**

Since the original version of this tutorial, the expected inputs for the spectral degradation function have changed, we must convert the previously loaded inputs to the expected form.  

We will created the desired spectra dataframe and conditions dataframe in the following cell.

In [12]:
# convert to expected format
spectra = SPECTRA["Spectra"]
spectra_df = pd.DataFrame(spectra.tolist(), index=spectra.index)
spectra_df = spectra.str.strip("[]").str.split(",", expand=True).astype(float)
spectra_df.columns = wavelengths

conditions_df = pd.DataFrame(
    index=spectra_df.index,
    data={
        "relative_humidity": SPECTRA["RH"],
        "temperature": SPECTRA["Temperature"],
    }
)

spectra_df

,280,300,320,340,360,380,400
timestamp,,,,,,,
2021-03-09 10:00:00,0.689215,0.402156,0.672589,0.026755,0.339772,0.943247,0.741140
2021-03-09 11:00:00,0.155757,0.546437,0.689580,0.782766,0.504982,0.933572,0.465200
2021-03-09 12:00:00,0.227821,0.905650,0.263872,0.057230,0.990580,0.936994,0.180034
2021-03-09 13:00:00,0.374194,0.035831,0.405237,0.957822,0.104399,0.891696,0.487553
2021-03-09 14:00:00,0.403212,0.647317,0.698112,0.063360,0.868030,0.686939,0.416581
2021-03-09 15:00:00,0.746134,0.838259,0.839097,0.134849,0.548454,0.606019,0.101388
2021-03-09 16:00:00,0.462911,0.032300,0.579419,0.370247,0.520598,0.566595,0.672994
2021-03-09 17:00:00,0.173503,0.813971,0.866274,0.344168,0.667745,0.689945,0.143764
2021-03-09 18:00:00,0.369919,0.887410,0.082397,0.085335,0.633560,0.195546,0.374416


### 2. Calculate Degradation

The spectral degradation function has several optional paramters. For more information, refer to the documentation. Below is a function call with the minimum required information.

In [ ]:
degradation = pvdeg.degradation.degradation(
    spectra_df=spectra_df,
    conditions_df=conditions_df
)
degradation

4.496994351804118e-38

In [15]:
degradation = pvdeg.degradation.degradation(spectra=SPECTRA['Spectra'],
                                            rh_module=SPECTRA['RH'],
                                            temp_module=SPECTRA['Temperature'],
                                            wavelengths=wavelengths)

degradation                                            

Removing brackets from spectral irradiance data
arr integral timestamp
2021-03-09 10:00:00     2.193695e-45
2021-03-09 11:00:00     1.833548e-46
2021-03-09 12:00:00     1.508690e-34
2021-03-09 13:00:00     4.256451e-47
2021-03-09 14:00:00     1.041473e-82
2021-03-09 15:00:00    5.746285e-189
2021-03-09 16:00:00     2.430948e-62
2021-03-09 17:00:00     1.440645e-46
2021-03-09 18:00:00     2.196529e-64
2021-03-09 19:00:00    1.433768e-103
Name: Arr_integrand, dtype: float64
wavelength integral timestamp
2021-03-09 10:00:00    0.000358
2021-03-09 11:00:00    0.000287
2021-03-09 12:00:00    0.000298
2021-03-09 13:00:00    0.000258
2021-03-09 14:00:00    0.000338
2021-03-09 15:00:00    0.000412
2021-03-09 16:00:00    0.000275
2021-03-09 17:00:00    0.000309
2021-03-09 18:00:00    0.000311
2021-03-09 19:00:00    0.000284
Name: G_integral, dtype: float64
delta degradation  timestamp
2021-03-09 10:00:00     7.861213e-49
2021-03-09 11:00:00     5.263019e-50
2021-03-09 12:00:00     4.496994e-38


C:\Users\tford\AppData\Local\Temp\2\ipykernel_26040\2414568496.py:1: DeprecationWarning: Call to deprecated function degradation (old double integral degradation function will be replaced 'pvdegradation' in an updated version of pvdeg).
  degradation = pvdeg.degradation.degradation(spectra=SPECTRA['Spectra'],


4.496994351804118e-38